In [ ]:
import database
import pandas as pd

voos = pd.read_sql( '''
select * from voos_sp_sj
where dt_partida_prevista >= '2018-12-30 00:00:00'
order by dt_partida_prevista''', database.engine )


In [ ]:

clima = pd.read_sql( """

with
	mappings as (
	select *
		from
			aero_ponto_coleta
		where
			exists(
				select
					1
					from
						voos_sp_sj voos
					where
						voos.id_aerodromo_origem = aero_ponto_coleta.id_aerodromo
				)


	)
  , resumo_clima as (
                    select
	                    ponto_id
	                  , date( dt_clima ) - interval 1 day as dt_clima
	                  , id_aerodromo
	                  , sum( precipitacao_total )         as precipitacao_total
	                  , avg( temperatura_ar )             as temperatura_ar
	                  , avg( vento_rajada_maxima )        as vento_rajada_maxima
	                    from
		                    clima
		                    inner join mappings using ( ponto_id )
	                    group by
		                    ponto_id
		                  , date( dt_clima )
                    )

select * from resumo_clima
                    """, database.engine )

In [ ]:
voos[ "atrasos_partida" ] = (voos[ "dt_partida_prevista" ] - voos[ "dt_partida_real" ]).dt.seconds / 60
voos[ "atrasos_chegada" ] = (voos[ "dt_chegada_prevista" ] - voos[ "dt_chegada_real" ]).dt.seconds / 60
# Estaremos preocupados apenas em prever os atrasos de partida.
# Um caso estranho que vemos aqui é que tanto atrasos_chegada e atrasos_partida tem um valor de minimo == 0.
# Não seria de se esperar que houvessem chegadas e partidas adiantadas?

In [ ]:
voos[ "atrasado" ] = voos[ "atrasos_partida" ] >= 20
voos[ "atrasado_previsto" ] = 0

In [ ]:
# Um ponto interessante de se considerar é a metrica utilizada para
# 'scorar' o modelo.
# Acuracia vs F1 vs Precisao são algumas possibilidades,
# E todas tem uma interpretação levemente diferente.

# Se pensarmos nas 4 possibilidades, vulgo:
# Dizemos que atrasa, mas ele não atrasa ( Falso positivo )
# Dizemos que atrasa, e de fato atrasa ( Verdadeiro Positivo)
# não Dizemos que atrasa, e ele atrasa ( Falso negativo)
# não Dizemos que atrasa, e ele não atrasa (Verdadeiro Negativo).

# Vamos pensar na idéia de enviar um ( push | email ) para o cliente, caso acharmos que o voo dele irá atrasar.
# Nesse caso,
# Falsos positivos são PÉSSIMOS. ( 1000x )
# Verdadeiros positivos são Bons. ( 100x )
# Falsos negativos é regular ( 10x )
# Verdadeiro positivo é regular ( 1x )
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

train_columns = [
		"nr_voo",
		"sg_di",
		# "id_tipo_linha",
		# "dt_partida_prevista",
		"id_empresa",
		"id_aerodromo_origem",
		"id_aerodromo_destino",
		]
df_y = voos[ [ "atrasado" ] ]
df_x = voos[ train_columns ]

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split( df_x, df_y, train_size = 0.8 )

print( "training accuracy_score: ", accuracy_score( df_train_y, [ 0 ] * df_train_y.shape[ 0 ] ) )
print( "training precision_score: ", precision_score( df_train_y, [ 0 ] * df_train_y.shape[ 0 ] ) )
print( "training f1_score: ", f1_score( df_train_y, [ 0 ] * df_train_y.shape[ 0 ] ) )
print( "training roc_auc_score: ", roc_auc_score( df_train_y, [ 0 ] * df_train_y.shape[ 0 ] ) )

print( "testing accuracy_score: ", accuracy_score( df_test_y, [ 0 ] * df_test_y.shape[ 0 ] ) )
print( "testing precision_score: ", precision_score( df_test_y, [ 0 ] * df_test_y.shape[ 0 ] ) )
print( "testing f1_score: ", f1_score( df_test_y, [ 0 ] * df_test_y.shape[ 0 ] ) )
print( "testing roc_auc_score: ", roc_auc_score( df_test_y, [ 0 ] * df_test_y.shape[ 0 ] ) )


In [ ]:
from category_encoders import TargetEncoder

# O que vem a ser id_justificativa?

# Usar as os pesos definidos como metricas
# Considerar precisão porque nossa principal preocupação são falsos positivos.


encoder = TargetEncoder(
		cols = train_columns,
		handle_unknown = 'value' ).fit( df_train_x, df_train_y )

df_train_x = encoder.transform( df_train_x )
df_test_x = encoder.transform( df_test_x )

In [ ]:
from sklearn.utils.testing import all_estimators
from sklearn.base import ClassifierMixin

classifiers = [ est for est in all_estimators() if issubclass( est[ 1 ], ClassifierMixin ) ]
print( classifiers )

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from lazypredict.Supervised import LazyClassifier, CLASSIFIERS

classifier = LazyClassifier( verbose = 1, predictions = True, ignore_warnings = False, classifiers = [ LogisticRegression, RandomForestClassifier, DecisionTreeClassifier,
                                                                                                       SGDClassifier ] )
classifier, predictions = classifier.fit( df_train_x, df_test_x, df_train_y, df_test_y )

In [ ]:
classifier

In [ ]:
print( "training accuracy_score: ", accuracy_score( df_train_y, df_predict_train_y ) )
print( "training precision_score: ", precision_score( df_train_y, df_predict_train_y ) )
print( "training f1_score: ", f1_score( df_train_y, df_predict_train_y ) )
print( "training roc_auc_score: ", roc_auc_score( df_train_y, df_predict_train_y ) )
print()
print( "testing accuracy_score: ", accuracy_score( df_test_y, df_predict_test_y ) )
print( "testing precision_score: ", precision_score( df_test_y, df_predict_test_y ) )
print( "testing f1_score: ", f1_score( df_test_y, df_predict_test_y ) )
print( "testing roc_auc_score: ", roc_auc_score( df_test_y, df_predict_test_y ) )
